In [6]:
#import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pymongo

### Initializing

In [24]:
# URL of page to be scraped
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

#begin with an open dictionary for all mars data
mars_data={}

### NASA Mars News

In [76]:
# Retrieve page with the requests module
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [77]:
# Examine the results, then determine element that contains sought info
soup = bs(response.text, 'html.parser')

In [80]:
# Grab 10 headlines
headings = soup.find_all('div', class_="content_title")
# A blank list to hold the headlines
headlines = []
# Loop over grabbed elements
for heading in headings:
    # If element has an a tag, get text
    headlines.append(heading.a.get_text(strip=True))                  

#grab the urls
news_urls = soup.find_all('div', class_="content_title")
for link in news_urls:
    news_links.append(("https://mars.nasa.gov")+link.a['href'])
print(news_links)

#grab the descriptions
descs = soup.find_all('div', class_="rollover_description_inner")
# A blank list to hold the descriptions
descriptions = []
# Loop over elements
for desc in descs:
    descriptions.append(desc.get_text(strip=True))

['https://mars.nasa.gov/news/8645/mars-helicopter-attached-to-nasas-perseverance-rover/', 'https://mars.nasa.gov/news/8641/nasas-perseverance-mars-rover-gets-its-wheels-and-air-brakes/', 'https://mars.nasa.gov/news/8634/109-million-names-now-aboard-nasas-perseverance-mars-rover/', 'https://mars.nasa.gov/news/8622/virginia-middle-school-student-earns-honor-of-naming-nasas-next-mars-rover/', 'https://mars.nasa.gov/news/8606/nasa-prepares-for-moon-and-mars-with-new-addition-to-its-deep-space-network/', 'https://mars.nasa.gov/news/8603/nasa-administrator-statement-on-moon-to-mars-initiative-fy-2021-budget/']


### JPL Mars Space Images - Featured Image

In [30]:
#visit the url
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

In [35]:
#examime the html
response = requests.get(url2)
soup = bs(response.text, 'html.parser')

In [36]:
#grab the images with the future url
big_images = soup.find_all('li', class_='slide')# A blank list to hold the headlines
featured_image_url = []

#append the beginning url to the image path
for big_image in big_images:
    try:
        featured_image_url.append(f"https://www.jpl.nasa.gov{big_image.a['data-fancybox-href']}")
    except:
        print("Scraping Complete! :)")

Scraping Complete! :)
Scraping Complete! :)
Scraping Complete! :)
Scraping Complete! :)
Scraping Complete! :)


### Mars Weather

In [38]:
#go to the url
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)

In [72]:
#examine the results
response3 = requests.get(url3)
soup3 = bs(response3.text, 'html.parser')

#find the items based on html
weather_updates=[]
tweet_times=[]
weather_tweets = soup3.find_all('div', class_='js-tweet-text-container')
weather_times = soup3.find_all('span', class_='_timestamp')

#grab the twets
for tweet in weather_tweets:
    try:
        weather_updates.append(tweet.p.get_text(strip=True))
        tweet.a.extract()
    except:
        print("Scraping Complete! :)")
        
#grab the timestamp        
for time in weather_times:
    try:
        tweet_times.append(time.get('data-time-ms'))
    except:
        print("Scraping Complete! :)")

Scraping Complete! :)
Scraping Complete! :)
Scraping Complete! :)
Scraping Complete! :)
Scraping Complete! :)


In [73]:
#convert timestamp to datetime
import datetime
tweet_times_dt =[]
for tweet in tweet_times:
    your_dt = datetime.datetime.fromtimestamp(int(tweet)/1000)
    tweet_times_dt.append(your_dt.strftime("%Y-%m-%d %H:%M:%S"))

#remove picture url from the tweet
weather_updates_fixed = []
for update in weather_updates:
    nopic = update.split('pic.twitter')[0]
    weather_updates_fixed.append(nopic)

### Mars Facts

In [45]:
#visit the url and grab the table
url="https://space-facts.com/mars/"
tables = pd.read_html(url)
tables
mars_df=tables[0]

In [46]:
mars_df.to_html('mars_data.html',index=False)

### Mars Hemispheres

In [47]:
#visit the mars hemispheres url
url4="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url4)

In [49]:
response4 = requests.get(url4)
soup4 = bs(response4.text, 'html.parser')

In [69]:
#looping through the individual image pages
import re
links = []
img_alt = []
image_alts=[]
for link in soup4.findAll('a',href=re.compile("enhanced")):
    try:
        links.append(link.get('href'))
    except:
        print("Scraping Complete! :)")
        
for name in soup4.findAll('img',alt=re.compile("Enhanced")):
    try:
        img_alt.append(name.get('alt'))
        for i in img_alt:
            i.replace(' Enhanced thumbnail', '')
    except:
        print("Scraping Complete! :)")  
for img in img_alt:
    img2 = img.replace(" Enhanced thumbnail",'')
    image_alts.append(img2)

In [67]:
#build the full link
final_links=[]

for link in links:
    final_links.append(f"https://astrogeology.usgs.gov{link}")

In [70]:
#loop through the four links and visit them all to pull image href
fullsize_image=[]
for link in final_links:

    response5 = requests.get(link)
    soup5 = bs(response5.text, 'html.parser')
    
    for image in soup5.find_all('a',href=re.compile("full.jpg")):
        try:
            fullsize_image.append(image.get('href'))
        except:
            print("Scraping Complete! :)")


In [63]:
#zip the url and title into one dictionary
hemisphere_image_urls = [{'title': a, 'img_url': f} for a, f in zip(image_alts, fullsize_image)]

In [66]:
#zip mars tweets into one dictionary
mars_tweets = [{'tweet': b, 'time': c} for b, c in zip(weather_updates_fixed, tweet_times_dt)]

In [79]:
mars_data["mars_news_headlines"] = headlines
mars_data["mars_news_desc"] = descriptions
mars_data["mars_news_links"] = news_links
mars_data["mars_image"] = featured_image_url
mars_data["mars_tweets"] = mars_tweets
mars_data["mars_hemispheres"] = hemisphere_image_urls

print(mars_data)

{'mars_news_headlines': ["Mars Helicopter Attached to NASA's Perseverance Rover", "NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes", "10.9 Million Names Now Aboard NASA's Perseverance Mars Rover", "Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover", 'NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network', 'NASA Administrator Statement on Moon to Mars Initiative, FY 2021 Budget'], 'mars_news_desc': ["The team also fueled the rover's sky crane to get ready for this summer's history-making launch.", 'After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.', "As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars.", 'NASA chose a seventh-grader from Virginia as winner of the agency\'s "Name the Rover" essay contest. Alexander